In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
# from tooling.enums import AssetClass, Index, Spot, StrikeSpread
# from tooling.fetch import fetch_option_data, fetch_spot_data
# from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [4]:
# async def get_expiry(f_today, index):

#     if index == 'bnf':    
#         if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
#             f_expiry = dt.date(2024, 1, 25)
#         elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
#             f_expiry = dt.date(2024, 1, 31)
#         elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
#             f_expiry = dt.date(2024, 2, 29)
#         elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
#             f_expiry = dt.date(2024, 2, 27)
#         elif f_today < dt.date(2023, 9, 1):
#             days_to_thursday = (3 - f_today.weekday()) % 7
#             nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
#             f_expiry = nearest_thursday
#             if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
#                 f_expiry = nearest_thursday - dt.timedelta(days=1)
#         elif f_today >= dt.date(2023, 9, 1):
#             if f_today.day < 24:
#                 days_to_wednesday = (2 - f_today.weekday()) % 7
#                 nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
#                 f_expiry = nearest_wednesday
#                 if nse.valid_days(
#                     start_date=nearest_wednesday, end_date=nearest_wednesday
#                 ).empty:
#                     f_expiry = nearest_wednesday - dt.timedelta(days=1)
#             else:
#                 days_to_thursday = (3 - f_today.weekday()) % 7
#                 nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
#                 f_expiry = nearest_thursday
#                 if nse.valid_days(
#                     start_date=nearest_thursday, end_date=nearest_thursday
#                 ).empty:
#                     f_expiry = nearest_thursday - dt.timedelta(days=1)
#         return f_expiry

#     elif index == 'nifty':
#         days_to_thursday = (3 - f_today.weekday()) % 7
#         nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
#         f_expiry = nearest_thursday
#         if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
#             f_expiry = nearest_thursday - dt.timedelta(days=1)
#         return f_expiry

#     elif index == 'finnifty' or index == 'fnf':
#         days_to_thursday = (1 - f_today.weekday()) % 7
#         nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
#         f_expiry = nearest_thursday
#         if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
#             f_expiry = nearest_thursday - dt.timedelta(days=1)
#         return f_expiry

#     elif index == 'midcpnifty' or index == 'midcp':
#         days_to_thursday = (0 - f_today.weekday()) % 7
#         nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
#         f_expiry = nearest_thursday
#         if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
#             f_expiry = nearest_thursday - dt.timedelta(days=1)
#         return f_expiry

# def get_option_contract_name(symbol, strike, expiry, opt_type):
#     temp = "0"
#     mth = expiry.month

#     if (expiry + dt.timedelta(days=7)).month != expiry.month:
#         date_string = expiry.strftime("%y%b").upper()
#         return f"{symbol}{date_string}{strike}{opt_type}"
#     else:
#         if expiry.day <= 9:
#             date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
#         else:
#             date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
#         return f"{symbol}{date_string}{strike}{opt_type}"

In [5]:
import calendar
import datetime as dt

def get_monthly_expiry(input_date, index):
    # Get the year and month from the input date
    year = input_date.year
    month = input_date.month

    # Find the last day of the month
    last_day_of_month = calendar.monthrange(year, month)[1]
    last_date_of_month = dt.datetime(year, month, last_day_of_month)
    print(last_date_of_month)

    if index == 'bnf' or index == 'banknifty':
        if input_date < dt.date(2024, 3, 31):
            while last_date_of_month.weekday() != 3:  # 3 corresponds to Thursday
                last_date_of_month -= dt.timedelta(days=1)
            if nse.valid_days(start_date=last_date_of_month, end_date=last_date_of_month).empty:
                last_date_of_month -= dt.timedelta(days=1)
            return last_date_of_month
        else:
            while last_date_of_month.weekday() != 2: # 2 corresponds to Wednesday
                last_date_of_month -= dt.timedelta(days=1)
            if nse.valid_days(start_date=last_date_of_month, end_date=last_date_of_month).empty:
                last_date_of_month -= dt.timedelta(days=1)
            return last_date_of_month

    elif index == 'nifty' or index == 'nf':
        while last_date_of_month.weekday() != 3:  # 3 corresponds to Thursday
            last_date_of_month -= dt.timedelta(days=1)
        if nse.valid_days(start_date=last_date_of_month, end_date=last_date_of_month).empty:
            last_date_of_month -= dt.timedelta(days=1)
        return last_date_of_month

    elif index == 'finnifty' or index == 'fin' or index == 'fnf':
        while last_date_of_month.weekday() != 1:  # 1 corresponds to Tuesday
            last_date_of_month -= dt.timedelta(days=1)
        if nse.valid_days(start_date=last_date_of_month, end_date=last_date_of_month).empty:
            last_date_of_month -= dt.timedelta(days=1)
        return last_date_of_month

    elif index == 'midcpnifty' or index == 'midcp':
        while last_date_of_month.weekday() != 0:  # 0 corresponds to Monday
            last_date_of_month -= dt.timedelta(days=1)
        if nse.valid_days(start_date=last_date_of_month, end_date=last_date_of_month).empty:
            last_date_of_month -= dt.timedelta(days=1)
        return last_date_of_month

    elif index == 'bsx' or index == 'sensex':
        while last_date_of_month.weekday() != 4:  # 4 corresponds to Friday
            last_date_of_month -= dt.timedelta(days=1)
        if nse.valid_days(start_date=last_date_of_month, end_date=last_date_of_month).empty:
            last_date_of_month -= dt.timedelta(days=1)
        print(f'Returning Expiry as : {last_date_of_month}')
        return last_date_of_month

In [6]:
def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = '0'
    mth=expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime('%y%b').upper()
        return f'{symbol}{date_string}{strike}{opt_type}'
    else:
        if expiry.day<=9:
            date_string = f'{expiry.year - 2000}{mth}{temp}{expiry.day}'
        else :
            date_string = f'{expiry.year - 2000}{mth}{expiry.day}'
        return f'{symbol}{date_string}{strike}{opt_type}'

In [7]:
# bnf_pandas = pd.read_csv('../data/nifty_1hr_tv (2).csv')
# bnf_pandas = pd.read_csv('../data/bnf_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/midcp_select_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/sensex_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/crude_4hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/gold_4hr_tv.csv')
# bnf_1min = pd.read_csv('../data/bnf_min.csv')
bnf_1min = pd.read_csv("../data/bnf.csv")
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"]).dt.tz_localize(None)
# bnf_1min = bnf_1min[((bnf_1min['datetime'].dt.year == 2020) & (bnf_1min['datetime'].dt.month == 4))]
bnf_1min = bnf_1min[
    (bnf_1min["datetime"].dt.year >= 2017) & (bnf_1min["datetime"].dt.year <= 2024)
]

In [8]:
portfolio_value = 10_00_000
index_leverage = 7
index_name = "BANKNIFTY"
index_str_for_opt = "bnf"
strike_spread = 100

In [9]:
bnf_1min.head()

,datetime,open,high,low,close,volume
0,2017-01-02 09:15:00,18242.3000,18248.2000,18175.9000,18181.2000,0
1,2017-01-02 09:16:00,18181.8500,18194.7000,18179.9500,18184.4500,0
2,2017-01-02 09:17:00,18184.9500,18189.2500,18133.8000,18133.8000,0
3,2017-01-02 09:18:00,18135.1000,18141.5500,18118.5500,18138.9500,0
4,2017-01-02 09:19:00,18138.9500,18142.5500,18120.4500,18124.3000,0


In [10]:
def calculate_monthly_ranges(df):
    # Ensure the 'datetime' column is in the correct format
    if 'datetime' in df.columns:
        df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
        df.set_index('datetime', inplace=True)
    
    # Ensure the index is a DatetimeIndex
    df.index = pd.to_datetime(df.index, errors='coerce')

    # Drop any rows with missing/NaT dates
    df = df.dropna(subset=['open', 'high', 'low', 'close'])

    # Resample to monthly data to get the high, low, open, and close for each month
    monthly_ranges = df.resample("ME").agg({
        "open": "first",    # First open of the month
        "high": "max",      # Maximum high of the month
        "low": "min",       # Minimum low of the month
        "close": "last"     # Last close of the month
    })
    monthly_ranges['datetime'] = monthly_ranges.index

    # print(monthly_ranges)
    return monthly_ranges

bnf_monthly = calculate_monthly_ranges(bnf_1min)
bnf_monthly

,open,high,low,close,datetime
datetime,,,,,
2017-01-31,18242.3000,19794.9000,17831.7500,19491.4000,2017-01-31
2017-02-28,19522.4000,21031.6500,19471.4000,20609.1500,2017-02-28
2017-03-31,20698.9000,21690.9000,20423.7000,21430.9500,2017-03-31
2017-04-30,21539.2000,22382.0500,21390.2500,22349.1500,2017-04-30
2017-05-31,22412.5500,23469.4500,22240.6000,23452.4000,2017-05-31
2017-06-30,23394.1000,23896.9000,22998.0500,23241.9500,2017-06-30
2017-07-31,23240.3500,25134.2000,23144.5000,25091.5500,2017-07-31
2017-08-31,25152.8500,25196.4500,23854.6000,24301.3500,2017-08-31
2017-09-30,24389.2000,25104.9000,23613.2000,24026.4000,2017-09-30


In [17]:
async def find_breakouts(df, monthly_ranges):

    results = []
    multiplier_to_range = 0.35

    monthly_ranges["datetime"] = pd.to_datetime(monthly_ranges["datetime"], errors='coerce')
    df['datetime'] = df.index
    df["datetime"] = pd.to_datetime(df["datetime"], errors='coerce')
    
    # monthly_ranges = monthly_ranges[:-2]
    # print(monthly_ranges.tail())
    # print(df.tail())

    dates_to_avoid = []

    for i in range(1, len(monthly_ranges)):
        # print(monthly_ranges.iloc[i])
        previous_month = monthly_ranges.iloc[i - 1]
        current_month = monthly_ranges.iloc[i]
        # print("Current month : ", current_month["datetime"].date())
        if current_month["datetime"].date() in dates_to_avoid:
            print("Date Avoided")
            continue
        previous_month_high = monthly_ranges.iloc[i - 1]["high"]
        previous_month_low = monthly_ranges.iloc[i - 1]["low"]
        previous_month_close = monthly_ranges.iloc[i - 1]["close"]

        current_month_timestamp = monthly_ranges.index[i]

        current_month_open = monthly_ranges.iloc[i]["open"]

        # Filter the 1-minute data for the current month
        # current_month_data = df[(df.index > monthly_ranges.index[i-1]) & (df.index <= current_month)]
        current_month_data = df.loc[(df["datetime"].dt.month == current_month_timestamp.month) & (df["datetime"].dt.year == current_month_timestamp.year)]
        # print('Current Month Data Tail')
        # print(current_month_data.tail())
        # print('.........................................')
        # print("Current month Data:\n", current_week_data)

        monthly_range = previous_month_high - previous_month_low
        addition_range = multiplier_to_range * monthly_range

        # Check for breakouts
        high_level = current_month_data["open"].iloc[0] + addition_range
        low_level = current_month_data["open"].iloc[0] - addition_range

        # high_level = current_week_data['open'].iloc[0] + addition_range
        # low_level = current_week_data['open'].iloc[0] - addition_range
        # print(current_week)
        # print("High Level:\n", high_level, "\nLow Level:\n", low_level)

        breakout_high = current_month_data[current_month_data["high"] >= high_level]
        breakout_low = current_month_data[current_month_data["low"] <= low_level]
        # print(high_level)
        # print(breakout_high)

        current_month_date = pd.to_datetime(current_month['datetime']).date()
        search_expiry_date = current_month_date - dt.timedelta(days=6)
        
        if not breakout_high.empty:
            # print('Breakout High')
            breakout_high_time = breakout_high.iloc[0]["datetime"]
            # print('High Break At :', breakout_high_time)
            atm_strike = int(round(low_level / strike_spread) * strike_spread)
            # exit_date = current_week['datetime'].date() + dt.timedelta(days=5)
            expiry = get_monthly_expiry(
                search_expiry_date,
                index_str_for_opt,
            ).date()
            dte = (expiry - breakout_high_time.date()).days
            contract = get_option_contract_name(
                symbol=index_name,
                strike=atm_strike,
                expiry=expiry,
                opt_type="PE",
            )
            pe_df = await fetch_data(
                index=index_str_for_opt,
                start_date=breakout_high_time.date(),
                end_date=expiry,
                start_time=breakout_high_time.time(),
                end_time=dt.time(15, 20),
                expiry=expiry,
                strike=atm_strike,
                asset_class="P",
            )
            if not isinstance(pe_df, str) and pe_df is not None:
                pe_df = pe_df.to_pandas()
                entry_price = pe_df.iloc[0]["c"]
            else:
                entry_price = float("nan")

            # Exit Scenario
            current_month_data_after_entry = current_month_data[
                (current_month_data["datetime"] > breakout_high_time)
                & (
                    current_month_data["datetime"].dt.date
                    <= expiry - dt.timedelta(days=0)
                )
            ]
            # print(current_month_data_after_entry)
            low_breach = current_month_data_after_entry[
                current_month_data_after_entry["low"] < previous_month_low
            ]
            # print(low_breach)
            if len(low_breach) != 0:
                exit_time = low_breach.iloc[0]["datetime"]
                # print(exit_time)
                remark = "SL Hit"
                # print(remark, exit_time)
                # print(pe_df)
                if not math.isnan(entry_price):
                    # print((pe_df[(pe_df['datetime'].dt.date == exit_time.date()) & (pe_df['datetime'].dt.hour >= exit_time.hour) & (pe_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    filtered_df = pe_df.loc[
                        (pe_df["datetime"].dt.date >= exit_time.date())
                        & (pe_df["datetime"].dt.hour >= exit_time.hour)
                        & (pe_df["datetime"].dt.minute >= exit_time.minute),
                        "c"]
                    exit_price = filtered_df.iloc[0] if not filtered_df.empty else float('nan')

                else:
                    exit_price = float("nan")
                    # print(exit_price)
            else:
                exit_time = dt.datetime.combine(
                    expiry - dt.timedelta(days=0), dt.time(15, 30)
                )
                remark = "Expiry Closing"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    exit_price = pe_df.iloc[-1]["c"]
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            # print(exit_time)
            # print(exit_price)
            # print(contract)
            qty = portfolio_value * index_leverage / atm_strike
            slippage = 0.01 * (entry_price + exit_price)
            pnl = qty * (entry_price - exit_price)
            final_pnl = qty * (entry_price - exit_price - slippage)
            time_diff = exit_time - breakout_high_time
            if not (time_diff == pd.Timedelta(minutes=1)):
                unit = {
                    "Month": previous_month["datetime"].date(),
                    "Month High": previous_month_high,
                    "Month Low": previous_month_low,
                    "Monthly Range": monthly_range,
                    "Current Month Open": current_month_open,
                    "Multiplier to Range": f"{int(multiplier_to_range * 100)}%",
                    "Entry Level Long": high_level,
                    "Entry Level Short": low_level,
                    "Break Type": "high",
                    "Strike": atm_strike,
                    "Expiry": expiry,
                    "DTE": dte,
                    "Contract": contract,
                    "Option Type": contract[-2:],
                    "Entry Time": breakout_high_time,
                    "Entry Price": entry_price,
                    "Exit Time": exit_time,
                    "Exit Price": exit_price,
                    "Remark": remark,
                    "Points": entry_price - exit_price,
                    "Slippage": slippage,
                    "Points w cs": (entry_price - exit_price) - slippage,
                    "Qty": qty,
                    "PnL": pnl,
                    "PnL w cs": final_pnl,
                    "ROI%": (pnl * 100 / portfolio_value),
                    "ROI% w cs": (final_pnl * 100 / portfolio_value),
                }
                # print(unit)
                results.append(unit)

        if not breakout_low.empty:
            # print('Breakout Low')
            breakout_low_time = breakout_low.iloc[0]["datetime"]
            # print('Low Break At :', breakout_low_time)
            atm_strike = int(round(high_level / strike_spread) * strike_spread)
            # exit_date = current_week['datetime'].date() + dt.timedelta(days=5)
            expiry = get_monthly_expiry(
                search_expiry_date,
                index_str_for_opt,
            ).date()
            dte = (expiry - breakout_low_time.date()).days
            contract = get_option_contract_name(
                symbol=index_name,
                strike=atm_strike,
                expiry=expiry,
                opt_type="CE",
            )
            ce_df = await fetch_data(
                index=index_str_for_opt,
                start_date=breakout_low_time.date(),
                end_date=expiry,
                start_time=breakout_low_time.time(),
                end_time=dt.time(15, 20),
                expiry=expiry,
                strike=atm_strike,
                asset_class="C",
            )
            # print(len(ce_df))
            if not isinstance(ce_df, str) and ce_df is not None:
                ce_df = ce_df.to_pandas()
                entry_price = ce_df.iloc[0]["c"]
            else:
                entry_price = float("nan")

            # Exit Scenario
            current_month_data_after_entry = current_month_data[
                (current_month_data["datetime"] > breakout_low_time)
                & (
                    current_month_data["datetime"].dt.date
                    <= expiry - dt.timedelta(days=0)
                )
            ]
            # print(current_week_data_after_entry)
            high_breach = current_month_data_after_entry[
                current_month_data_after_entry["high"] > previous_month_high
            ]
            # print(high_breach)
            if len(high_breach) != 0:
                exit_time = high_breach.iloc[0]["datetime"]
                # print(exit_time)
                remark = "SL Hit"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    # print(exit_time.date(), exit_time.hour, exit_time.minute)
                    # print((ce_df[(ce_df['datetime'].dt.date == exit_time.date()) & (ce_df['datetime'].dt.hour >= exit_time.hour) & (ce_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    filtered_df = ce_df.loc[
                        (ce_df["datetime"].dt.date >= exit_time.date())
                        & (ce_df["datetime"].dt.hour >= exit_time.hour)
                        & (ce_df["datetime"].dt.minute >= exit_time.minute),
                        "c"]
                    exit_price = filtered_df.iloc[0] if not filtered_df.empty else float('nan') 
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            else:
                exit_time = dt.datetime.combine(
                    expiry - dt.timedelta(days=0), dt.time(15, 30)
                )
                remark = "Expiry Closing"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    exit_price = ce_df.iloc[-1]["c"]
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            # print(exit_time)
            # print(exit_price)
            # print(contract)

            qty = portfolio_value * index_leverage / atm_strike
            slippage = 0.01 * (entry_price + exit_price)
            pnl = qty * (entry_price - exit_price)
            final_pnl = qty * (entry_price - exit_price - slippage)
            time_diff = exit_time - breakout_low_time
            if not (time_diff == pd.Timedelta(minutes=1)):
                unit = {
                    "Month": previous_month["datetime"].date(),
                    "Month High": previous_month_high,
                    "Month Low": previous_month_low,
                    "Monthly Range": monthly_range,
                    "Current Month Open": current_month_open,
                    "Multiplier to Range": f"{int(multiplier_to_range * 100)}%",
                    "Entry Level Long": high_level,
                    "Entry Level Short": low_level,
                    "Break Type": "low",
                    "Strike": atm_strike,
                    "Expiry": expiry,
                    "DTE": dte,
                    "Contract": contract,
                    "Option Type": contract[-2:],
                    "Entry Time": breakout_low_time,
                    "Entry Price": entry_price,
                    "Exit Time": exit_time,
                    "Exit Price": exit_price,
                    "Remark": remark,
                    "Points": entry_price - exit_price,
                    "Slippage": slippage,
                    "Points w cs": (entry_price - exit_price) - slippage,
                    "Qty": qty,
                    "PnL": pnl,
                    "PnL w cs": final_pnl,
                    "ROI%": (pnl * 100 / portfolio_value),
                    "ROI% w cs": (final_pnl * 100 / portfolio_value),
                }
                # print(unit)
                results.append(unit)

    # print(results)
    return results

In [18]:
async def trade():
    df = bnf_1min
    # df.index = pd.to_datetime(df.index)
    # df['datetime'] = df.index.date
    # print(df.head())
    monthly_ranges = bnf_monthly
    breakouts = await find_breakouts(df, monthly_ranges)
    breakouts_pandas = pd.DataFrame(breakouts)
    # breakouts_polars = pl.DataFrame(breakouts)
    return breakouts_pandas


tradebook = await trade()

2017-02-28 00:00:00
2017-03-31 00:00:00
2017-04-30 00:00:00
2017-05-31 00:00:00
2017-06-30 00:00:00
2017-07-31 00:00:00
2017-08-31 00:00:00
2017-09-30 00:00:00
2017-09-30 00:00:00
2017-10-31 00:00:00
2017-11-30 00:00:00
2017-12-31 00:00:00
2017-12-31 00:00:00
2018-01-31 00:00:00
2018-02-28 00:00:00
2018-03-31 00:00:00
2018-04-30 00:00:00
2018-05-31 00:00:00
2018-06-30 00:00:00
2018-07-31 00:00:00
2018-08-31 00:00:00
2018-09-30 00:00:00
2018-11-30 00:00:00
2018-12-31 00:00:00
2019-01-31 00:00:00
2019-02-28 00:00:00
2019-03-31 00:00:00
2019-05-31 00:00:00
2019-05-31 00:00:00
2019-06-30 00:00:00
2019-07-31 00:00:00
2019-08-31 00:00:00
2019-09-30 00:00:00
2019-10-31 00:00:00
2019-11-30 00:00:00
2019-12-31 00:00:00
2020-01-31 00:00:00
2020-02-29 00:00:00
2020-02-29 00:00:00
2020-03-31 00:00:00
2020-05-31 00:00:00
2020-06-30 00:00:00
2020-07-31 00:00:00
2020-08-31 00:00:00
2020-09-30 00:00:00
2020-10-31 00:00:00
2020-11-30 00:00:00
2021-01-31 00:00:00
2021-01-31 00:00:00
2021-02-28 00:00:00


In [19]:
# tradebook = tradebook.replace(float('nan'), np.nan)
# tradebook = tradebook.dropna(subset=['ROI% w cs'])
# tradebook

In [20]:
tradebook

,Month,Month High,Month Low,Monthly Range,Current Month Open,Multiplier to Range,Entry Level Long,Entry Level Short,Break Type,Strike,Expiry,DTE,Contract,Option Type,Entry Time,Entry Price,Exit Time,Exit Price,Remark,Points,Slippage,Points w cs,Qty,PnL,PnL w cs,ROI%,ROI% w cs
0,2017-01-31,19794.9000,17831.7500,1963.1500,19522.4000,35%,20209.5025,18835.2975,high,18800,2017-02-23,20,BANKNIFTY17FEB18800PE,PE,2017-02-03 14:13:00,24.2500,2017-02-23 15:30:00,0.0500,Expiry Closing,24.2000,0.2430,23.9570,372.3404,9010.6383,8920.1596,0.9011,0.8920
1,2017-02-28,21031.6500,19471.4000,1560.2500,20698.9000,35%,21244.9875,20152.8125,high,20200,2017-03-30,16,BANKNIFTY17MAR20200PE,PE,2017-03-14 09:15:00,38.0000,2017-03-30 15:30:00,0.0500,Expiry Closing,37.9500,0.3805,37.5695,346.5347,13150.9901,13019.1337,1.3151,1.3019
2,2017-03-31,21690.9000,20423.7000,1267.2000,21539.2000,35%,21982.7200,21095.6800,high,21100,2017-04-27,2,BANKNIFTY17APR21100PE,PE,2017-04-25 12:41:00,3.8000,2017-04-27 15:30:00,0.0500,Expiry Closing,3.7500,0.0385,3.7115,331.7536,1244.0758,1231.3033,0.1244,0.1231
3,2017-04-30,22382.0500,21390.2500,991.8000,22412.5500,35%,22759.6800,22065.4200,high,22100,2017-05-25,20,BANKNIFTY17MAY22100PE,PE,2017-05-05 09:15:00,103.7000,2017-05-25 15:30:00,0.1500,Expiry Closing,103.5500,1.0385,102.5115,316.7421,32798.6425,32469.7059,3.2799,3.2470
4,2017-05-31,23469.4500,22240.6000,1228.8500,23394.1000,35%,23824.1975,22964.0025,high,23000,2017-06-29,7,BANKNIFTY17JUN23000PE,PE,2017-06-22 10:42:00,7.3500,2017-06-29 15:30:00,0.0500,Expiry Closing,7.3000,0.0740,7.2260,304.3478,2221.7391,2199.2174,0.2222,0.2199
5,2017-06-30,23896.9000,22998.0500,898.8500,23240.3500,35%,23554.9475,22925.7525,high,22900,2017-07-27,17,BANKNIFTY17JUL22900PE,PE,2017-07-10 09:15:00,123.1500,2017-07-27 15:30:00,0.0500,Expiry Closing,123.1000,1.2320,121.8680,305.6769,37628.8210,37252.2271,3.7629,3.7252
6,2017-07-31,25134.2000,23144.5000,1989.7000,25152.8500,35%,25849.2450,24456.4550,low,25800,2017-08-31,22,BANKNIFTY17AUG25800CE,CE,2017-08-09 09:20:00,22.2000,2017-08-31 15:30:00,0.0500,Expiry Closing,22.1500,0.2225,21.9275,271.3178,6009.6899,5949.3217,0.6010,0.5949
7,2017-08-31,25196.4500,23854.6000,1341.8500,24389.2000,35%,24858.8475,23919.5525,high,23900,2017-09-28,15,BANKNIFTY17SEP23900PE,PE,2017-09-13 10:02:00,31.0500,2017-09-27 14:08:00,111.9000,SL Hit,-80.8500,1.4295,-82.2795,292.8870,-23679.9163,-24098.5983,-2.3680,-2.4099
8,2017-08-31,25196.4500,23854.6000,1341.8500,24389.2000,35%,24858.8475,23919.5525,low,24900,2017-09-28,1,BANKNIFTY17SEP24900CE,CE,2017-09-27 12:33:00,1.6000,2017-09-28 15:30:00,0.1000,Expiry Closing,1.5000,0.0170,1.4830,281.1245,421.6867,416.9076,0.0422,0.0417
9,2017-09-30,25104.9000,23613.2000,1491.7000,24200.6000,35%,24722.6950,23678.5050,high,23700,2017-10-26,13,BANKNIFTY17OCT23700PE,PE,2017-10-13 13:09:00,26.5500,2017-10-26 15:30:00,0.0500,Expiry Closing,26.5000,0.2660,26.2340,295.3586,7827.0042,7748.4388,0.7827,0.7748


In [21]:
# x = tradebook['Entry Time'].iloc[0].year
tradebook["Entry Time"] = pd.to_datetime(tradebook["Entry Time"])
tradebook["Trade Year"] = tradebook["Entry Time"].dt.year

In [22]:
stats_df8 = pd.DataFrame(
    index=range(2017, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tradebook
# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["ROI% w cs"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["ROI% w cs"] >= 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["ROI% w cs"] > 0]["ROI% w cs"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["ROI% w cs"] < 0]["ROI% w cs"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["ROI% w cs"].cumsum() - year_trades["ROI% w cs"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df8["Total ROI"].sum()
overall_total_trades = stats_df8["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["ROI% w cs"] >= 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% w cs"] >= 0][
    "ROI% w cs"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% w cs"] < 0][
    "ROI% w cs"
].mean()
overall_max_drawdown = (
    combined_df_sorted["ROI% w cs"].cumsum()
    - combined_df_sorted["ROI% w cs"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df8

/tmp/ipykernel_51325/2761328021.py:40: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,19.2342,13,92.3077,1.8037,-2.4099,-2.4099,7.9814
2018,18.9476,11,90.9091,1.9091,-0.1436,-0.1436,131.9580
2019,13.7653,12,83.3333,2.4771,-5.5026,-9.8866,1.3923
2020,43.6549,11,90.9091,4.4505,-0.8505,-0.8505,51.3278
2021,30.9863,14,100.0000,2.2133,NaN,0.0000,inf
2022,17.0811,13,84.6154,1.5571,-0.0470,-0.0470,363.6022
2023,7.4225,13,76.9231,1.6965,-9.5428,-9.5428,0.7778
2024,4.9365,5,80.0000,1.2341,NaN,0.0000,inf
Overall,156.0285,92.0000,88.0435,2.2226,-3.4284,-9.8866,15.7818


# BNF Monthly with 8x Lev RBOS

In [24]:
# tradebook.to_csv('monthly bnf rbos.csv', index=False)

In [609]:
# Add Hedges

def round_off(value, index):
    if index == 'bnf':
        return 100 * round(value / 100)
    else:
        return 50 * round(value / 50)

async def add_hedges(df):
    # df.drop(columns=['ROI%', 'Trade Year'], inplace=True)
    contract = df.iloc[0]['Contract']
    hedge_pct = 4
    index_name = "BANKNIFTY" if 'BANK' in contract else ('FINNIFTY' if 'FIN' in contract else 'NIFTY') 
    index_str_for_opt = "bnf" if 'BANK' in contract else ('finnifty' if 'FIN' in contract else 'nifty')

    # df['Hedge Strike'] = df.apply(lambda row: row['Strike'] + round_to_nearest_100((hedge_pct / 100) * row['Strike']) if 'C' in row['Contract'] else row['Strike'] - round_to_nearest_100((hedge_pct / 100) * row['Strike']), axis=1)
    df["Hedge Strike"] = df.apply(
        lambda row: (
            round_off(row["Strike"] * (1 + hedge_pct / 100), index_str_for_opt)
            if row["Option Type"] == "CE"
            else round_off(row["Strike"] * (1 - hedge_pct / 100), index_str_for_opt)
        ),
        axis=1,
    )

    df["Hedge Contract"] = df.apply(
        lambda row: get_option_contract_name(
            index_name, row["Hedge Strike"], row["Expiry"], row["Option Type"]
        ),
        axis=1,
    )

    df["Hedge Entry Price"] = np.nan
    df["Hedge Exit Price"] = np.nan

    for i in range(0, len(df)):
        print(df.iloc[i]["Month"])
        hedge_strike = int(df.iloc[i]["Hedge Strike"])
        hedge_expiry = df.iloc[i]["Expiry"]
        hedge_opt_type = df.iloc[i]["Option Type"][-2:-1]
        hedge_entry_datetime = df.iloc[i]["Entry Time"]
        hedge_exit_datetime = df.iloc[i]["Exit Time"]

        hedge_df = await fetch_data(
            index=index_str_for_opt,
            start_date=hedge_entry_datetime.date(),
            end_date=hedge_exit_datetime.date(),
            start_time=hedge_entry_datetime.time(),
            end_time=hedge_exit_datetime.time(),
            expiry=hedge_expiry,
            strike=hedge_strike,
            asset_class=hedge_opt_type,
        )
        if not isinstance(hedge_df, str) and hedge_df is not None:
            hedge_df = hedge_df.to_pandas()
        else:
            print(hedge_df)
            df.loc[i, "Hedge Entry Price"] = 0
            df.loc[i, "Hedge Exit Price"] = 0
            continue
        # print(hedge_df)

        hedge_entry_price = hedge_df.iloc[0]["c"]
        hedge_exit_price = hedge_df.iloc[-1]["c"]

        print(hedge_entry_price, hedge_exit_price)

        df.loc[i, "Hedge Entry Price"] = hedge_entry_price
        df.loc[i, "Hedge Exit Price"] = hedge_exit_price

    return df


tb_with_hedge = await add_hedges(tradebook)
# tb_with_hedge


2017-01-31
6.85 0.05
2017-02-28
7.1 0.05
2017-03-31
0.9 0.05
2017-03-31
0.95 0.7
2017-04-30
6.2 0.05
2017-04-30
1.65 1.55
2017-05-31
9.0 0.05
2017-05-31
0.45 0.05
2017-06-30
None
2017-07-31
2.45 0.05
2017-08-31
8.15 0.1
2017-08-31
0.7 0.05
2017-09-30
5.25 0.05
2017-10-31
2.7 0.55
2017-11-30
14.6 25.85
2017-11-30
7.5 2.15
2017-12-31
9.6 0.05
2017-12-31
2.65 2.6
2018-01-31
5.0 0.05
2018-02-28
3.7 0.05
2018-03-31
19.4 0.1
2018-04-30
6.4 0.05
2018-04-30
3.15 4.0
2018-05-31
10.1 0.05
2018-05-31
3.9 0.05
2018-06-30
11.55 0.05
2018-06-30
4.1 5.0
2018-07-31
3.65 0.2
2018-08-31
5.3 0.05
2018-09-30
9.35 0.05
2018-10-31
11.7 0.05
2018-11-30
10.35 1.65
2018-12-31
10.95 4.35
2019-01-31
27.55 0.05
2019-01-31
4.6 0.05
2019-02-28
22.9 0.05
2019-03-31
2.0 0.05
2019-04-30
101.25 176.4
2019-04-30
90.0 90.0
2019-05-31
0.15 0.15
2019-06-30
18.95 34.55
2019-06-30
5.0 0.85
2019-07-31
6.0 0.05
2019-08-31
8.85 0.1
2019-08-31
13.05 14.35
2019-09-30
1.2 0.05
2019-09-30
19.95 11.3
2019-10-31
1.45 0.05
2019-11-30


In [610]:
tb_with_hedge["Qty"] = tb_with_hedge["Qty"] * 1.5
tb_with_hedge["Hedge Points"] = (
    tb_with_hedge["Hedge Exit Price"] - tb_with_hedge["Hedge Entry Price"]
)
tb_with_hedge["PnL"] = tb_with_hedge["Points"] * tb_with_hedge["Qty"]
tb_with_hedge["PnL w cs"] = tb_with_hedge["Points w cs"] * tb_with_hedge["Qty"]
tb_with_hedge["Hedge PnL"] = tb_with_hedge["Hedge Points"] * tb_with_hedge["Qty"]
tb_with_hedge["Total PnL"] = tb_with_hedge["PnL w cs"] + tb_with_hedge["Hedge PnL"]

In [611]:
tb_with_hedge["Total ROI%"] = tb_with_hedge["Total PnL"] * 100 / 1000000
tb_with_hedge["Entry Time"] = pd.to_datetime(tb_with_hedge["Entry Time"])
tb_with_hedge["Trade Year"] = tb_with_hedge["Entry Time"].dt.year

In [612]:
tb_with_hedge

,Month,Month High,Month Low,Monthly Range,Current Month Open,Multiplier to Range,Entry Level Long,Entry Level Short,Break Type,Strike,Expiry,DTE,Contract,Option Type,Entry Time,Entry Price,Exit Time,Exit Price,Remark,Points,Slippage,Points w cs,Qty,PnL,PnL w cs,ROI%,ROI% w cs,Trade Year,Hedge Strike,Hedge Contract,Hedge Entry Price,Hedge Exit Price,Hedge Points,Hedge PnL,Total PnL,Total ROI%
0,2017-01-31,8672.5500,8134.3000,538.2500,8570.3500,20%,8678.0000,8462.7000,high,8450,2017-02-23,22,NIFTY17FEB8450PE,PE,2017-02-01 13:59:00,34.4500,2017-02-23 15:30:00,0.0500,Expiry Closing,34.4000,0.3450,34.0550,1242.6036,42745.5621,42316.8639,2.8497,2.8211,2017,8100,NIFTY17FEB8100PE,6.8500,0.0500,-6.8000,-8449.7041,33867.1598,3.3867
1,2017-02-28,8981.9000,8537.8000,444.1000,8904.4000,20%,8993.2200,8815.5800,high,8800,2017-03-30,16,NIFTY17MAR8800PE,PE,2017-03-14 09:15:00,12.9500,2017-03-30 15:30:00,0.0500,Expiry Closing,12.9000,0.1300,12.7700,1193.1818,15392.0455,15236.9318,1.0261,1.0158,2017,8450,NIFTY17MAR8450PE,7.1000,0.0500,-7.0500,-8411.9318,6825.0000,0.6825
2,2017-03-31,9214.1000,8860.3000,353.8000,9220.6000,20%,9291.3600,9149.8400,high,9150,2017-04-27,2,NIFTY17APR9150PE,PE,2017-04-25 14:00:00,6.1500,2017-04-27 15:30:00,0.0500,Expiry Closing,6.1000,0.0620,6.0380,1147.5410,7000.0000,6928.8525,0.4667,0.4619,2017,8800,NIFTY17APR8800PE,0.9000,0.0500,-0.8500,-975.4098,5953.4426,0.5953
3,2017-03-31,9214.1000,8860.3000,353.8000,9220.6000,20%,9291.3600,9149.8400,low,9300,2017-04-27,14,NIFTY17APR9300CE,CE,2017-04-13 15:11:00,27.7000,2017-04-18 09:36:00,35.9500,SL Hit,-8.2500,0.6365,-8.8865,1129.0323,-9314.5161,-10033.1452,-0.6210,-0.6689,2017,9650,NIFTY17APR9650CE,0.9500,0.7000,-0.2500,-282.2581,-10315.4032,-1.0315
4,2017-04-30,9367.0500,9075.3000,291.7500,9339.8500,20%,9398.2000,9281.5000,high,9300,2017-05-25,15,NIFTY17MAY9300PE,PE,2017-05-10 14:49:00,36.9000,2017-05-25 15:30:00,0.0500,Expiry Closing,36.8500,0.3695,36.4805,1129.0323,41604.8387,41187.6613,2.7737,2.7458,2017,8950,NIFTY17MAY8950PE,6.2000,0.0500,-6.1500,-6943.5484,34244.1129,3.4244
5,2017-04-30,9367.0500,9075.3000,291.7500,9339.8500,20%,9398.2000,9281.5000,low,9400,2017-05-25,23,NIFTY17MAY9400CE,CE,2017-05-02 11:16:00,45.8500,2017-05-05 09:15:00,57.4500,SL Hit,-11.6000,1.0330,-12.6330,1117.0213,-12957.4468,-14111.3298,-0.8638,-0.9408,2017,9800,NIFTY17MAY9800CE,1.6500,1.5500,-0.1000,-111.7021,-14223.0319,-1.4223
6,2017-05-31,9649.3500,9270.0000,379.3500,9603.5500,20%,9679.4200,9527.6800,high,9550,2017-06-29,24,NIFTY17JUN9550PE,PE,2017-06-05 11:31:00,47.5000,2017-06-29 15:30:00,36.5000,Expiry Closing,11.0000,0.8400,10.1600,1099.4764,12094.2408,11170.6806,0.8063,0.7447,2017,9150,NIFTY17JUN9150PE,9.0000,0.0500,-8.9500,-9840.3141,1330.3665,0.1330
7,2017-05-31,9649.3500,9270.0000,379.3500,9603.5500,20%,9679.4200,9527.6800,low,9700,2017-06-29,2,NIFTY17JUN9700CE,CE,2017-06-27 10:31:00,4.5000,2017-06-29 15:30:00,0.0500,Expiry Closing,4.4500,0.0455,4.4045,1082.4742,4817.0103,4767.7577,0.3211,0.3179,2017,10100,NIFTY17JUN10100CE,0.4500,0.0500,-0.4000,-432.9897,4334.7680,0.4335
8,2017-06-30,9706.3000,9448.8000,257.5000,9587.9500,20%,9639.4500,9536.4500,high,9550,2017-07-27,23,NIFTY17JUL9550PE,PE,2017-07-04 09:15:00,73.2500,2017-07-27 15:30:00,0.0500,Expiry Closing,73.2000,0.7330,72.4670,1099.4764,80481.6754,79675.7592,5.3654,5.3117,2017,9150,NIFTY17JUL9150PE,0.0000,0.0000,0.0000,0.0000,79675.7592,7.9676
9,2017-07-31,10114.7500,9543.8500,570.9000,10101.0500,20%,10215.2300,9986.8700,low,10200,2017-08-31,23,NIFTY17AUG10200CE,CE,2017-08-08 10:14:00,33.5000,2017-08-31 15:30:00,0.0500,Expiry Closing,33.4500,0.3355,33.1145,1029.4118,34433.8235,34088.4559,2.2956,2.2726,2017,10600,NIFTY17AUG10600CE,2.4500,0.0500,-2.4000,-2470.5882,31617.8676,3.1618


In [615]:
tb_with_hedge['Hedge PnL'].sum()
tb_with_hedge['PnL w cs'].sum()

3019233.2419805205

In [613]:
stats_df8 = pd.DataFrame(
    index=range(2017, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tb_with_hedge
# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["Total ROI%"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["Total ROI%"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["Total ROI%"] > 0]["Total ROI%"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["Total ROI%"] < 0]["Total ROI%"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["Total ROI%"].cumsum() - year_trades["Total ROI%"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df8["Total ROI"].sum()
overall_total_trades = stats_df8["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["Total ROI%"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["Total ROI%"] > 0][
    "Total ROI%"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["Total ROI%"] < 0][
    "Total ROI%"
].mean()
overall_max_drawdown = (
    combined_df_sorted["Total ROI%"].cumsum()
    - combined_df_sorted["Total ROI%"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,5.5586,16,68.7500,2.2955,-3.9383,-12.1232,0.4585
2018,28.7788,16,75.0000,3.6441,-3.7377,-9.2845,3.0996
2019,7.2667,18,66.6667,2.9111,-4.6111,-14.7651,0.4922
2020,46.8355,15,80.0000,5.5211,-6.4726,-11.6707,4.0131
2021,37.7024,18,83.3333,3.6810,-5.8375,-11.8148,3.1911
2022,20.9031,16,87.5000,3.8384,-16.4171,-25.7560,0.8116
2023,22.3705,15,73.3333,3.9060,-6.8652,-8.9653,2.4952
2024,10.2667,11,54.5455,4.3166,-3.1266,-8.6511,1.1867
Overall,179.6823,125.0000,74.4000,3.7418,-5.4291,-25.7560,6.9763
